# 음성 모델 구축하기

이제 우리는 음성 데이터를 스펙트로그램 이미지를 만들었으니 컴퓨터 비전 모델을 구축해야 합니다. 학습 경로를 따르고 있다면 이미 경로의 두 번째 모듈에서 컴퓨터 비전 모델을 만들었습니다. 
비전 모델을 구축하기 위해 [vision](https://pypi.org/project/torchvision/) 패키지를 사용해야하 합니다. 모델을 구축하는 데 필요한 패키지를 가져오도록 합니다.

In [1]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import pandas as pd
import os

## 학습을 위해 DataLoader에 Spectrogram 이미지 불러오기

여기서는 이미지 데이터의 경로를 제공하고 `ImageFolder` helper를 사용하여 이미지를 텐서로 불러 옵니다. 라벨은 폴더 이름을 기준으로 구성되어 있습니다.

In [2]:
data_path = './data/spectrograms' #looking in subfolder train

yes_no_dataset = datasets.ImageFolder(
    root=data_path,
    transform=transforms.Compose([transforms.Resize((201,81)),
                                  transforms.ToTensor()
                                  ])
)
print(yes_no_dataset)
print(yes_no_dataset[5][0].size())

Dataset ImageFolder
    Number of datapoints: 7985
    Root location: ./data/spectrograms
    StandardTransform
Transform: Compose(
               Resize(size=(201, 81), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )
torch.Size([3, 201, 81])


## 훈련 및 테스트를 위해 데이터 분할

- 데이터를 분할하여 train, test set을 8:2로 나눕니다. 모델 학습에 80 %를 사용하고 모델 성능를 평가하는데 20 %를 사용합니다.

In [3]:
#split data to test and train
#use 80% to train
train_size = int(0.8 * len(yes_no_dataset))
test_size = len(yes_no_dataset) - train_size
yes_no_train_dataset, yes_no_test_dataset = torch.utils.data.random_split(yes_no_dataset, [train_size, test_size])

print(len(yes_no_train_dataset))
print(len(yes_no_test_dataset))

6388
1597


- `DataLoader`로 trian, test 데이터셋을 불러옵니다.

In [4]:
train_dataloader = torch.utils.data.DataLoader(
    yes_no_train_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    yes_no_test_dataset,
    batch_size=15,
    num_workers=2,
    shuffle=True
)

- DataLoader로 읽은 데이터셋의 텐서 형태를 살펴봅시다.

In [5]:
train_dataloader.dataset[0][0][0][0]

tensor([0.5608, 0.4824, 0.5098, 0.4353, 0.3490, 0.4824, 0.5843, 0.4392, 0.4667,
        0.3608, 0.4431, 0.5059, 0.4667, 0.4824, 0.4745, 0.4118, 0.4431, 0.2039,
        0.4510, 0.3765, 0.3804, 0.4118, 0.4745, 0.4706, 0.4392, 0.5020, 0.5020,
        0.5137, 0.5059, 0.3922, 0.3059, 0.3804, 0.4196, 0.3647, 0.3647, 0.4392,
        0.3451, 0.3804, 0.3608, 0.4078, 0.6392, 0.6980, 0.7529, 0.5333, 0.7137,
        0.8000, 0.5882, 0.6196, 0.7373, 0.6863, 0.6863, 0.6118, 0.5922, 0.4863,
        0.5725, 0.5608, 0.5098, 0.4745, 0.4745, 0.4431, 0.4706, 0.4000, 0.4039,
        0.4314, 0.4275, 0.3333, 0.1569, 0.4353, 0.4824, 0.4588, 0.4118, 0.3961,
        0.4314, 0.4118, 0.2667, 0.3686, 0.4980, 0.4941, 0.4275, 0.4863, 0.5922])

- 모델 학습을 위해 GPU를 가져오거나 GPU를 사용할 수 없는 경우 CPU를 사용합니다.

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))


Using cuda device


## 신경망 생성하기
- CNN(convolution neural network)을 만들고 장치(GPU or CPU)를 설정합니다.

In [7]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(51136, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)
    
model = CNNet().to(device)

print(model)

CNNet(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (flatten): Flatten()
  (fc1): Linear(in_features=51136, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
)


## 학습과 테스트 관련 함수 생성
- 여기서는 비용 함수(cost function),  learning_rate 및 optimizer을 설정합니다. 그런 다음 사용할 train, test 함수를 설정합니다.

In [8]:
# cost function used to determine best parameters
cost = torch.nn.CrossEntropyLoss()

# used to create optimal parameters
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create the training function

def train(dataloader, model, loss, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


# Create the validation/test function

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    print(f'\nTest Error:\nacc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}\n')

## 모델 학습하기
- 이제 epoch 수를 설정하고 epoch 마다 `train`과 `test`함수를 호출하도록 만듭니다.

In [9]:
epochs = 15

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, cost, optimizer)
    test(test_dataloader, model)
print('Done!')

Epoch 1
-------------------------------
loss: 0.683419  [    0/ 6388]
loss: 0.695789  [ 1500/ 6388]
loss: 0.692987  [ 3000/ 6388]
loss: 0.666870  [ 4500/ 6388]
loss: 0.692348  [ 6000/ 6388]

Test Error:
acc: 49.7%, avg loss: 0.046441

Epoch 2
-------------------------------
loss: 0.693147  [    0/ 6388]
loss: 0.693147  [ 1500/ 6388]
loss: 0.693147  [ 3000/ 6388]
loss: 0.693147  [ 4500/ 6388]
loss: 0.723022  [ 6000/ 6388]

Test Error:
acc: 49.7%, avg loss: 0.046441

Epoch 3
-------------------------------
loss: 0.693147  [    0/ 6388]
loss: 0.666938  [ 1500/ 6388]
loss: 0.773418  [ 3000/ 6388]
loss: 0.666485  [ 4500/ 6388]
loss: 0.685241  [ 6000/ 6388]

Test Error:
acc: 49.7%, avg loss: 0.046441

Epoch 4
-------------------------------
loss: 0.695083  [    0/ 6388]
loss: 0.706812  [ 1500/ 6388]
loss: 0.620915  [ 3000/ 6388]
loss: 0.488353  [ 4500/ 6388]
loss: 0.471648  [ 6000/ 6388]

Test Error:
acc: 86.9%, avg loss: 0.022696

Epoch 5
-------------------------------
loss: 0.271708  [   

## 모델 평가 하기

좋은 결과 입니다! 15번째 EPOCH까지는 93%-95%의 정확도를 얻었어야 합니다. 여기서는 테스트 데이터 세트의 결과를 가져와서 예측 라벨과 실제 라벨을 보고 모델의 성능이 어느정도 인지 확인합니다.

In [10]:
model.eval()
test_loss, correct = 0, 0

with torch.no_grad():
    for batch, (X, Y) in enumerate(test_dataloader):
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        print("Predicted:")
        print(f"{pred.argmax(1)}")
        print("Actual:")
        print(f"{Y}")
        break

Predicted:
tensor([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0], device='cuda:0')
Actual:
tensor([1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0], device='cuda:0')
